In [30]:
# Импорты и настройки
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

VALIDATION_SCROLL_SEGMENT = '20231210121321'  # Сегмент для валидации
DATA_USAGE_PERCENTAGE = 0.7
RANDOM_STATE = 42

In [31]:
def stratified_split_by_scroll_and_area(group, validation_segment, data_usage_percentage=DATA_USAGE_PERCENTAGE, test_size=0.15, random_state=42):
    """
    Разделяет данные на train и validation с учетом размера сегментов и стратификации.
    
    Args:
        group: DataFrame с данными одного свитка
        validation_segment: ID сегмента для валидации
        data_usage_percentage: Доля используемых данных (от 0 до 1)
        test_size: Доля валидационной выборки
        random_state: Seed для воспроизводимости
    """
    # Создаем копию данных во избежание предупреждений
    group = group.copy()
    
    # Применяем data_usage_percentage ко всем данным, кроме validation_segment
    validation_mask = group['segment'] == validation_segment
    validation_segment_row = group[validation_mask]
    remaining_group = group[~validation_mask]
    
    # Уменьшаем размер выборки согласно data_usage_percentage
    if data_usage_percentage < 1.0:
        remaining_group = remaining_group.sample(
            frac=data_usage_percentage, 
            random_state=random_state
        ).reset_index(drop=True)

    # Добавление диапазонов площади
    bins = [0, 10, 50, float('inf')]  # Маленькие, средние, большие
    labels = ['small', 'medium', 'large']
    remaining_group.loc[:, 'area_range'] = pd.cut(
        remaining_group['area_cm2'], 
        bins=bins, 
        labels=labels
    )

    # Разделение оставшихся данных по диапазонам площади
    train_parts = []
    validation_parts = []
    
    # Используем observed=True для устранения предупреждения
    for area_range, subgroup in remaining_group.groupby('area_range', observed=True):
        if len(subgroup) == 0:
            continue

        try:
            train, val = train_test_split(
                subgroup,
                test_size=test_size,
                stratify=subgroup[['gp_labels', 'nicola_labels']],
                random_state=random_state
            )
        except ValueError:
            train = subgroup
            val = pd.DataFrame(columns=subgroup.columns)

        train_parts.append(train)
        validation_parts.append(val)

    # Объединение частей с явным указанием игнорирования индекса
    train = pd.concat(train_parts, ignore_index=True) if train_parts else pd.DataFrame(columns=group.columns)
    validation = pd.concat(validation_parts, ignore_index=True) if validation_parts else pd.DataFrame(columns=group.columns)

    # Добавление валидационного сегмента
    if not validation_segment_row.empty:
        validation = pd.concat([validation, validation_segment_row], ignore_index=True)

    return train, validation

In [32]:
def load_and_prepare_data(filepath):
    """Загружает и фильтрует данные."""
    data = pd.read_csv(filepath)
    return data[(data['gp_labels'] == True) | (data['nicola_labels'] == True)]

In [33]:
data_path = "/home/jovyan/Bulygin/villa-main/ink-detection/segments_info.csv"
data = pd.read_csv(data_path)
filtered_data = load_and_prepare_data(data_path)
filtered_data

,scroll,segment,area_cm2,gp_labels,nicola_labels
0,1,20231022170901,134.591000,True,True
1,1,20231210121321,107.610000,True,True
2,1,20231221180251,73.730600,True,True
3,1,20230530164535,4.464850,True,False
4,1,20231007101615,77.745100,True,False
...,...,...,...,...,...
64,5,20241108115232,18.528183,False,True
65,5,20241108120732,20.318417,False,True
66,5,20241113070770,9.372654,False,True
67,5,20241113080880,9.430482,False,True


In [34]:
# Группировка по свиткам
grouped_by_scroll = filtered_data.groupby('scroll')
final_train = []
final_validation = []

for scroll, group in grouped_by_scroll:
    train, validation = stratified_split_by_scroll_and_area(
        group, 
        VALIDATION_SCROLL_SEGMENT,
        data_usage_percentage=DATA_USAGE_PERCENTAGE
    )
    final_train.append(train)
    final_validation.append(validation)

# Объединение результатов с игнорированием индекса
train_df = pd.concat(final_train, ignore_index=True) if final_train else pd.DataFrame(columns=filtered_data.columns)
validation_df = pd.concat(final_validation, ignore_index=True) if final_validation else pd.DataFrame(columns=filtered_data.columns)

print("Исходный размер данных:", len(data))
print("Размер после фильтрации:", len(filtered_data))
print("Размер тренировочной выборки:", len(train_df) + len(validation_df))


train_df.to_csv("train.csv", index=False)
validation_df.to_csv("validation.csv", index=False)

Исходный размер данных: 85
Размер после фильтрации: 69
Размер тренировочной выборки: 48
